In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

# Load the preprocessed dataset
df_qos = pd.read_csv("zoom480p_preprocessed.csv")

# Display dataset structure
df_qos.info()
df_qos.head()
    

In [ ]:

# Function to classify adaptive bitrate based on QoS conditions
def classify_bitrate(row):
    if row["RTT (ping) (ms)"] < 50 and row["Rx packet loss (percent)"] < 1:
        return "1080p"
    elif 50 <= row["RTT (ping) (ms)"] <= 100 or 1 <= row["Rx packet loss (percent)"] <= 3:
        return "720p"
    elif row["RTT (ping) (ms)"] > 100 or row["Rx packet loss (percent)"] > 3:
        return "480p"
    else:
        return "360p"

# Apply classification
df_qos["Adaptive Bitrate"] = df_qos.apply(classify_bitrate, axis=1)

# Display updated dataset
df_qos.head()
    

In [ ]:

# Select relevant features for predicting Adaptive Bitrate
features = ["RTT (ping) (ms)", "Rx packet loss (percent)", "Tx packet loss (percent)", 
            "Rx instant jitter (ms)", "Tx instant jitter (ms)", "Download (Rx) bandwidth (Mbps)", 
            "Upload (Tx) bandwidth (Mbps)"]

X = df_qos[features]
y = df_qos["Adaptive Bitrate"]

# Encode Adaptive Bitrate labels (convert categorical to numerical values)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Display dataset shape
X_train.shape, X_test.shape, y_train.shape, y_test.shape
    

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on test data
y_pred = rf_model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f"Model Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_rep)
    

In [ ]:

import pickle

# Save the trained model
with open("adaptive_bitrate_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

# Save the label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("Model and encoder saved successfully.")
    

In [ ]:

from flask import Flask, request, jsonify
import pickle
import numpy as np

# Load trained model and encoder
with open("adaptive_bitrate_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

app = Flask(__name__)

@app.route("/predict_bitrate", methods=["POST"])
def predict_bitrate():
    data = request.json
    qos_features = np.array([[
        data["RTT (ping) (ms)"], data["Rx packet loss (percent)"], data["Tx packet loss (percent)"],
        data["Rx instant jitter (ms)"], data["Tx instant jitter (ms)"], 
        data["Download (Rx) bandwidth (Mbps)"], data["Upload (Tx) bandwidth (Mbps)"]
    ]])

    # Predict bitrate
    predicted_label = model.predict(qos_features)[0]
    predicted_bitrate = label_encoder.inverse_transform([predicted_label])[0]

    return jsonify({"Recommended Bitrate": predicted_bitrate})

if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=5002)
    